In [61]:
import pandas as pd 
import numpy as np 
import requests
import bs4 as bs
import urllib.request 

## Extracting the features of 2020 movies from Wikipedia

In [62]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"

In [63]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml') 

In [64]:
tables = soup.find_all('table',class_='wikitable sortable')

In [65]:
len(tables)

4

In [66]:
type(tables[0])

bs4.element.Tag

In [67]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [68]:
p1 = pd.concat([df4,df3],axis=0,ignore_index=True)
p2 = pd.concat([p1,df2],axis=0,ignore_index=True)
df = pd.concat([p2,df1],axis=0,ignore_index=True)

In [69]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.
0,O C T O B E R,2,On the Rocks,Apple TV+ / A24 / American Zoetrope,Sofia Coppola (director/screenplay); Rashida J...,[168],NaN
1,O C T O B E R,2,Spontaneous,Paramount Pictures / Jurassic Party Productions,Brian Duffield (director/screenplay); Katherin...,[169],NaN
2,O C T O B E R,2,Vampires vs. the Bronx,Netflix / Universal Pictures,Osmany Rodriguez (director/screenplay); Blaise...,[170],NaN
3,O C T O B E R,2,Possessor,Neon / Well Go USA Entertainment / Elevation P...,Brandon Cronenberg (director/screenplay); Andr...,[171],NaN
4,O C T O B E R,2,Death of Me,Saban Films,"Darren Lynn Bousman (director); Arli Margolis,...",[172],NaN
...,...,...,...,...,...,...,...
270,M A R C H,13,Lost Transmissions,Gravitas Ventures,Katharine O'Brien (director/screenplay); Simon...,NaN,[53]
271,M A R C H,20,No films had been released on this date as res...,No films had been released on this date as res...,No films had been released on this date as res...,NaN,No films had been released on this date as res...
272,M A R C H,27,Resistance,IFC Films,Jonathan Jakubowicz (director/screenplay); Jes...,NaN,[54]
273,M A R C H,27,Uncorked,Netflix,Prentice Penny (director/screenplay); Mamoudou...,NaN,[55]


In [70]:
df_2020 = df[['Title','Cast and crew']]

In [71]:
df_2020

,Title,Cast and crew
0,On the Rocks,Sofia Coppola (director/screenplay); Rashida J...
1,Spontaneous,Brian Duffield (director/screenplay); Katherin...
2,Vampires vs. the Bronx,Osmany Rodriguez (director/screenplay); Blaise...
3,Possessor,Brandon Cronenberg (director/screenplay); Andr...
4,Death of Me,"Darren Lynn Bousman (director); Arli Margolis,..."
...,...,...
270,Lost Transmissions,Katharine O'Brien (director/screenplay); Simon...
271,No films had been released on this date as res...,No films had been released on this date as res...
272,Resistance,Jonathan Jakubowicz (director/screenplay); Jes...
273,Uncorked,Prentice Penny (director/screenplay); Mamoudou...


In [72]:
!pip install tmdbv3api

In [73]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key='729fcde16e57db670079f06107ea69bc'

In [74]:
from tmdbv3api import Movie

# Create a Movie object
tmdb_movie = Movie()

# Function to get genre from movie title
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
        return np.NaN
    else:
        try:
            movie_id = result[0].id
            response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id, tmdb_movie.api_key))
            data_json = response.json()
            if 'genres' in data_json and data_json['genres']:
                genre_str = " " 
                for genre in data_json['genres']:
                    genres.append(genre['name'])
                return genre_str.join(genres)
            else:
                return np.NaN
        except Exception as e:
            print("Error:", e)
            return np.NaN


In [75]:
for index, row in df_2020.iterrows():
    df_2020.at[index, 'genres'] = get_genre(str(row['Title']))

/tmp/ipykernel_3070/413768800.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020.at[index, 'genres'] = get_genre(str(row['Title']))


Error: getattr(): attribute name must be string
Error: getattr(): attribute name must be string


In [76]:
df_2020

,Title,Cast and crew,genres
0,On the Rocks,Sofia Coppola (director/screenplay); Rashida J...,Drama Comedy
1,Spontaneous,Brian Duffield (director/screenplay); Katherin...,Comedy Horror
2,Vampires vs. the Bronx,Osmany Rodriguez (director/screenplay); Blaise...,Comedy Horror
3,Possessor,Brandon Cronenberg (director/screenplay); Andr...,Horror Thriller Science Fiction
4,Death of Me,"Darren Lynn Bousman (director); Arli Margolis,...",Horror Mystery
...,...,...,...
270,Lost Transmissions,Katharine O'Brien (director/screenplay); Simon...,Drama
271,No films had been released on this date as res...,No films had been released on this date as res...,NaN
272,Resistance,Jonathan Jakubowicz (director/screenplay); Jes...,War History Drama
273,Uncorked,Prentice Penny (director/screenplay); Mamoudou...,Drama Comedy


In [77]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [78]:
df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(str(x)))

/tmp/ipykernel_3070/3848435694.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(str(x)))


In [79]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [80]:
df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))

/tmp/ipykernel_3070/3646981363.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))


In [81]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [82]:
df_2020['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor2(str(x)))

/tmp/ipykernel_3070/463054939.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor2(str(x)))


In [83]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [84]:
df_2020['actor_3_name'] = df_2020['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [85]:
df_2020

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,On the Rocks,Sofia Coppola (director/screenplay); Rashida J...,Drama Comedy,Sofia Coppola,Rashida Jones,Bill Murray,Marlon Wayans
1,Spontaneous,Brian Duffield (director/screenplay); Katherin...,Comedy Horror,Brian Duffield,Katherine Langford,Charlie Plummer,Hayley Law
2,Vampires vs. the Bronx,Osmany Rodriguez (director/screenplay); Blaise...,Comedy Horror,Osmany Rodriguez,Jaden Michael,Gerald W. Jones III,Gregory Diaz IV
3,Possessor,Brandon Cronenberg (director/screenplay); Andr...,Horror Thriller Science Fiction,Brandon Cronenberg,Andrea Riseborough,Christopher Abbott,Rossif Sutherland
4,Death of Me,"Darren Lynn Bousman (director); Arli Margolis,...",Horror Mystery,Darren Lynn Bousman,Maggie Q,Alex Essoe,Luke Hemsworth
...,...,...,...,...,...,...,...
270,Lost Transmissions,Katharine O'Brien (director/screenplay); Simon...,Drama,Katharine O'Brien,Simon Pegg,Juno Temple,Alexandra Daddario
271,No films had been released on this date as res...,No films had been released on this date as res...,NaN,No films had been released on this date as res...,No films had been released on this date as res...,NaN,NaN
272,Resistance,Jonathan Jakubowicz (director/screenplay); Jes...,War History Drama,Jonathan Jakubowicz,Jesse Eisenberg,Ed Harris,Clémence Poésy
273,Uncorked,Prentice Penny (director/screenplay); Mamoudou...,Drama Comedy,Prentice Penny,Mamoudou Athie,Courtney B. Vance,Niecy Nash


In [86]:
df_2020 = df_2020.rename(columns={'Title':'movie_title'})

In [87]:
new_df20 = df_2020.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [88]:
new_df20

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Sofia Coppola,Rashida Jones,Bill Murray,Marlon Wayans,Drama Comedy,On the Rocks
1,Brian Duffield,Katherine Langford,Charlie Plummer,Hayley Law,Comedy Horror,Spontaneous
2,Osmany Rodriguez,Jaden Michael,Gerald W. Jones III,Gregory Diaz IV,Comedy Horror,Vampires vs. the Bronx
3,Brandon Cronenberg,Andrea Riseborough,Christopher Abbott,Rossif Sutherland,Horror Thriller Science Fiction,Possessor
4,Darren Lynn Bousman,Maggie Q,Alex Essoe,Luke Hemsworth,Horror Mystery,Death of Me
...,...,...,...,...,...,...
270,Katharine O'Brien,Simon Pegg,Juno Temple,Alexandra Daddario,Drama,Lost Transmissions
271,No films had been released on this date as res...,No films had been released on this date as res...,NaN,NaN,NaN,No films had been released on this date as res...
272,Jonathan Jakubowicz,Jesse Eisenberg,Ed Harris,Clémence Poésy,War History Drama,Resistance
273,Prentice Penny,Mamoudou Athie,Courtney B. Vance,Niecy Nash,Drama Comedy,Uncorked


In [89]:
new_df20['comb'] = new_df20['actor_1_name'] + ' ' + new_df20['actor_2_name'] + ' '+ new_df20['actor_3_name'] + ' '+ new_df20['director_name'] +' ' + new_df20['genres']

In [90]:
new_df20.isna().sum()

director_name     0
actor_1_name      0
actor_2_name      5
actor_3_name     28
genres            2
movie_title       0
comb             29
dtype: int64

In [91]:
new_df20 = new_df20.dropna(how='any')

In [92]:
new_df20.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [93]:
new_df20['movie_title'] = new_df20['movie_title'].str.lower()

/tmp/ipykernel_3070/2267385682.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df20['movie_title'] = new_df20['movie_title'].str.lower()


In [94]:
new_df20

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Sofia Coppola,Rashida Jones,Bill Murray,Marlon Wayans,Drama Comedy,on the rocks,Rashida Jones Bill Murray Marlon Wayans Sofia ...
1,Brian Duffield,Katherine Langford,Charlie Plummer,Hayley Law,Comedy Horror,spontaneous,Katherine Langford Charlie Plummer Hayley Law ...
2,Osmany Rodriguez,Jaden Michael,Gerald W. Jones III,Gregory Diaz IV,Comedy Horror,vampires vs. the bronx,Jaden Michael Gerald W. Jones III Gregory Diaz...
3,Brandon Cronenberg,Andrea Riseborough,Christopher Abbott,Rossif Sutherland,Horror Thriller Science Fiction,possessor,Andrea Riseborough Christopher Abbott Rossif S...
4,Darren Lynn Bousman,Maggie Q,Alex Essoe,Luke Hemsworth,Horror Mystery,death of me,Maggie Q Alex Essoe Luke Hemsworth Darren Lynn...
...,...,...,...,...,...,...,...
268,Eliza Hittman,Sidney Flanigan,Talia Ryder,Theodore Pellerin,Drama,never rarely sometimes always,Sidney Flanigan Talia Ryder Theodore Pellerin ...
269,Jason Orley,Pete Davidson,Griffin Gluck,Jon Cryer,Comedy,big time adolescence,Pete Davidson Griffin Gluck Jon Cryer Jason Or...
270,Katharine O'Brien,Simon Pegg,Juno Temple,Alexandra Daddario,Drama,lost transmissions,Simon Pegg Juno Temple Alexandra Daddario Kath...
272,Jonathan Jakubowicz,Jesse Eisenberg,Ed Harris,Clémence Poésy,War History Drama,resistance,Jesse Eisenberg Ed Harris Clémence Poésy Jonat...


## Extracting the features of 2021 movies from Wikipedia

In [95]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2021"

In [96]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [97]:
tables = soup.find_all('table',class_='wikitable sortable')

In [98]:
len(tables)

4

In [99]:
type(tables[0])

bs4.element.Tag

In [100]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [101]:
p1=pd.concat([df4,df3],axis=0,ignore_index=True)
p2=pd.concat([p1,df2],axis=0,ignore_index=True)
df=pd.concat([p2,df1],axis=0,ignore_index=True)

In [102]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.
0,O C T O B E R,1.0,Venom: Let There Be Carnage,Columbia Pictures / Marvel Entertainment / Ten...,"Andy Serkis (director), Kelly Marcel (screenpl...",[203],NaN
1,O C T O B E R,1.0,The Many Saints of Newark,Warner Bros. Pictures / New Line Cinema / HBO ...,"Alan Taylor (director); David Chase, Lawrence ...",[190],NaN
2,O C T O B E R,1.0,The Addams Family 2,Metro-Goldwyn-Mayer / Bron Creative,"Conrad Vernon, Greg Tiernan (directors); Dan H...",[204],NaN
3,O C T O B E R,1.0,Bingo Hell,Amazon Studios / Blumhouse Television,Gigi Saul Guerrero (director/screenplay); Shan...,[205],NaN
4,O C T O B E R,1.0,Black as Night,Amazon Studios / Blumhouse Television,Maritte Lee Go (director); Sherman Payne (scre...,[205],NaN
...,...,...,...,...,...,...,...
360,M A R C H,26.0,Bad Trip,Netflix / Orion Pictures,Kitao Sakurai (director/screenplay); Dan Curry...,NaN,NaN
361,M A R C H,26.0,The Seventh Day,Vertical Entertainment / Voltage Pictures,Justin P. Lange (director/screenplay); Guy Pea...,NaN,[55]
362,M A R C H,26.0,Senior Moment,Screen Media Films,"Giorgio Serafini (director); Kurt Brungardt, C...",NaN,[56]
363,M A R C H,26.0,North Hollywood,Illegal Civilization,Mikey Alfred (director); Rusty Johnson (screen...,NaN,[57]


In [103]:
df_2021 = df[['Title','Cast and crew']]

In [104]:
df_2021

,Title,Cast and crew
0,Venom: Let There Be Carnage,"Andy Serkis (director), Kelly Marcel (screenpl..."
1,The Many Saints of Newark,"Alan Taylor (director); David Chase, Lawrence ..."
2,The Addams Family 2,"Conrad Vernon, Greg Tiernan (directors); Dan H..."
3,Bingo Hell,Gigi Saul Guerrero (director/screenplay); Shan...
4,Black as Night,Maritte Lee Go (director); Sherman Payne (scre...
...,...,...
360,Bad Trip,Kitao Sakurai (director/screenplay); Dan Curry...
361,The Seventh Day,Justin P. Lange (director/screenplay); Guy Pea...
362,Senior Moment,"Giorgio Serafini (director); Kurt Brungardt, C..."
363,North Hollywood,Mikey Alfred (director); Rusty Johnson (screen...


In [105]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '729fcde16e57db670079f06107ea69bc'

In [106]:
for index, row in df_2021.iterrows():
    df_2021.at[index, 'genres'] = get_genre(str(row['Title']))

/tmp/ipykernel_3070/1035393997.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021.at[index, 'genres'] = get_genre(str(row['Title']))


Error: getattr(): attribute name must be string


In [107]:
df_2021

,Title,Cast and crew,genres
0,Venom: Let There Be Carnage,"Andy Serkis (director), Kelly Marcel (screenpl...",Science Fiction Action Adventure
1,The Many Saints of Newark,"Alan Taylor (director); David Chase, Lawrence ...",Crime Drama
2,The Addams Family 2,"Conrad Vernon, Greg Tiernan (directors); Dan H...",Family Animation Comedy Fantasy Adventure
3,Bingo Hell,Gigi Saul Guerrero (director/screenplay); Shan...,Thriller Horror
4,Black as Night,Maritte Lee Go (director); Sherman Payne (scre...,Horror Thriller
...,...,...,...
360,Bad Trip,Kitao Sakurai (director/screenplay); Dan Curry...,Comedy
361,The Seventh Day,Justin P. Lange (director/screenplay); Guy Pea...,Horror
362,Senior Moment,"Giorgio Serafini (director); Kurt Brungardt, C...",Comedy Romance
363,North Hollywood,Mikey Alfred (director); Rusty Johnson (screen...,Comedy Drama


In [108]:
df_2021['director_name'] = df_2021['Cast and crew'].map(lambda x: get_director(str(x)))

/tmp/ipykernel_3070/3118254134.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['director_name'] = df_2021['Cast and crew'].map(lambda x: get_director(str(x)))


In [109]:
df_2021['actor_1_name'] = df_2021['Cast and crew'].map(lambda x: get_actor1(str(x)))

/tmp/ipykernel_3070/1611661901.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['actor_1_name'] = df_2021['Cast and crew'].map(lambda x: get_actor1(str(x)))


In [110]:
df_2021['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))

/tmp/ipykernel_3070/4272336897.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))


In [111]:
df_2021['actor_3_name'] = df_2021['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [112]:
df_2021

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,Venom: Let There Be Carnage,"Andy Serkis (director), Kelly Marcel (screenpl...",Science Fiction Action Adventure,Andy Serkis,Tom Hardy,Rashida Jones,Naomie Harris
1,The Many Saints of Newark,"Alan Taylor (director); David Chase, Lawrence ...",Crime Drama,Alan Taylor,Alessandro Nivola,Katherine Langford,Leslie Odom Jr.
2,The Addams Family 2,"Conrad Vernon, Greg Tiernan (directors); Dan H...",Family Animation Comedy Fantasy Adventure,"Conrad Vernon, Greg Tiernan",Oscar Isaac,Jaden Michael,Chloë Grace Moretz
3,Bingo Hell,Gigi Saul Guerrero (director/screenplay); Shan...,Thriller Horror,Gigi Saul Guerrero,L. Scott Caldwell,Andrea Riseborough,Joshua Caleb Johnson
4,Black as Night,Maritte Lee Go (director); Sherman Payne (scre...,Horror Thriller,Maritte Lee Go,Ashja Cooper,Maggie Q,Kenneisha Thompson
...,...,...,...,...,...,...,...
360,Bad Trip,Kitao Sakurai (director/screenplay); Dan Curry...,Comedy,Kitao Sakurai,Eric André,NaN,Tiffany Haddish
361,The Seventh Day,Justin P. Lange (director/screenplay); Guy Pea...,Horror,Justin P. Lange,Guy Pearce,NaN,Stephen Lang
362,Senior Moment,"Giorgio Serafini (director); Kurt Brungardt, C...",Comedy Romance,Giorgio Serafini,William Shatner,NaN,Christopher Lloyd
363,North Hollywood,Mikey Alfred (director); Rusty Johnson (screen...,Comedy Drama,Mikey Alfred,Ryder McLaughlin,NaN,Miranda Cosgrove


In [113]:
df_2021 = df_2021.rename(columns={'Title':'movie_title'})

In [114]:
new_df21 = df_2020.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [115]:
new_df21

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Sofia Coppola,Rashida Jones,Bill Murray,Marlon Wayans,Drama Comedy,On the Rocks
1,Brian Duffield,Katherine Langford,Charlie Plummer,Hayley Law,Comedy Horror,Spontaneous
2,Osmany Rodriguez,Jaden Michael,Gerald W. Jones III,Gregory Diaz IV,Comedy Horror,Vampires vs. the Bronx
3,Brandon Cronenberg,Andrea Riseborough,Christopher Abbott,Rossif Sutherland,Horror Thriller Science Fiction,Possessor
4,Darren Lynn Bousman,Maggie Q,Alex Essoe,Luke Hemsworth,Horror Mystery,Death of Me
...,...,...,...,...,...,...
270,Katharine O'Brien,Simon Pegg,Juno Temple,Alexandra Daddario,Drama,Lost Transmissions
271,No films had been released on this date as res...,No films had been released on this date as res...,NaN,NaN,NaN,No films had been released on this date as res...
272,Jonathan Jakubowicz,Jesse Eisenberg,Ed Harris,Clémence Poésy,War History Drama,Resistance
273,Prentice Penny,Mamoudou Athie,Courtney B. Vance,Niecy Nash,Drama Comedy,Uncorked


In [116]:
new_df21['comb'] = new_df21['actor_1_name'] + ' ' + new_df21['actor_2_name'] + ' '+ new_df21['actor_3_name'] + ' '+ new_df21['director_name'] +' ' + new_df21['genres']

In [117]:
new_df21.isna().sum()

director_name     0
actor_1_name      0
actor_2_name      5
actor_3_name     28
genres            2
movie_title       0
comb             29
dtype: int64

In [118]:
new_df21 = new_df21.dropna(how='any')

In [119]:
new_df20.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [120]:
new_df21['movie_title'] = new_df21['movie_title'].str.lower()

/tmp/ipykernel_3070/2732271610.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df21['movie_title'] = new_df21['movie_title'].str.lower()


In [121]:
new_df21

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Sofia Coppola,Rashida Jones,Bill Murray,Marlon Wayans,Drama Comedy,on the rocks,Rashida Jones Bill Murray Marlon Wayans Sofia ...
1,Brian Duffield,Katherine Langford,Charlie Plummer,Hayley Law,Comedy Horror,spontaneous,Katherine Langford Charlie Plummer Hayley Law ...
2,Osmany Rodriguez,Jaden Michael,Gerald W. Jones III,Gregory Diaz IV,Comedy Horror,vampires vs. the bronx,Jaden Michael Gerald W. Jones III Gregory Diaz...
3,Brandon Cronenberg,Andrea Riseborough,Christopher Abbott,Rossif Sutherland,Horror Thriller Science Fiction,possessor,Andrea Riseborough Christopher Abbott Rossif S...
4,Darren Lynn Bousman,Maggie Q,Alex Essoe,Luke Hemsworth,Horror Mystery,death of me,Maggie Q Alex Essoe Luke Hemsworth Darren Lynn...
...,...,...,...,...,...,...,...
268,Eliza Hittman,Sidney Flanigan,Talia Ryder,Theodore Pellerin,Drama,never rarely sometimes always,Sidney Flanigan Talia Ryder Theodore Pellerin ...
269,Jason Orley,Pete Davidson,Griffin Gluck,Jon Cryer,Comedy,big time adolescence,Pete Davidson Griffin Gluck Jon Cryer Jason Or...
270,Katharine O'Brien,Simon Pegg,Juno Temple,Alexandra Daddario,Drama,lost transmissions,Simon Pegg Juno Temple Alexandra Daddario Kath...
272,Jonathan Jakubowicz,Jesse Eisenberg,Ed Harris,Clémence Poésy,War History Drama,resistance,Jesse Eisenberg Ed Harris Clémence Poésy Jonat...


In [128]:
df_21_20 = pd.concat([df_2020,df_2021],axis=0,ignore_index=True)

In [129]:
df_21_20

,movie_title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,On the Rocks,Sofia Coppola (director/screenplay); Rashida J...,Drama Comedy,Sofia Coppola,Rashida Jones,Bill Murray,Marlon Wayans
1,Spontaneous,Brian Duffield (director/screenplay); Katherin...,Comedy Horror,Brian Duffield,Katherine Langford,Charlie Plummer,Hayley Law
2,Vampires vs. the Bronx,Osmany Rodriguez (director/screenplay); Blaise...,Comedy Horror,Osmany Rodriguez,Jaden Michael,Gerald W. Jones III,Gregory Diaz IV
3,Possessor,Brandon Cronenberg (director/screenplay); Andr...,Horror Thriller Science Fiction,Brandon Cronenberg,Andrea Riseborough,Christopher Abbott,Rossif Sutherland
4,Death of Me,"Darren Lynn Bousman (director); Arli Margolis,...",Horror Mystery,Darren Lynn Bousman,Maggie Q,Alex Essoe,Luke Hemsworth
...,...,...,...,...,...,...,...
635,Bad Trip,Kitao Sakurai (director/screenplay); Dan Curry...,Comedy,Kitao Sakurai,Eric André,NaN,Tiffany Haddish
636,The Seventh Day,Justin P. Lange (director/screenplay); Guy Pea...,Horror,Justin P. Lange,Guy Pearce,NaN,Stephen Lang
637,Senior Moment,"Giorgio Serafini (director); Kurt Brungardt, C...",Comedy Romance,Giorgio Serafini,William Shatner,NaN,Christopher Lloyd
638,North Hollywood,Mikey Alfred (director); Rusty Johnson (screen...,Comedy Drama,Mikey Alfred,Ryder McLaughlin,NaN,Miranda Cosgrove


In [130]:
old_df = pd.read_csv('final_data.csv')

In [131]:
old_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
5848,Jeff Tremaine,Douglas Booth,Iwan Rheon,Machine Gun Kelly,Drama Music History Comedy,the dirt,Douglas Booth Iwan Rheon Machine Gun Kelly Jef...
5849,S. Craig Zahler,Mel Gibson,Vince Vaughn,Michael Jai White,Crime Action Thriller,dragged across concrete,Mel Gibson Vince Vaughn Michael Jai White S. C...
5850,Tim Burton,Colin Farrell,Michael Keaton,Danny DeVito,Animation Family,dumbo,Colin Farrell Michael Keaton Danny DeVito Tim ...
5851,Harmony Korine,Matthew McConaughey,Zac Efron,Jonah Hill,Comedy,the beach bum,Matthew McConaughey Zac Efron Jonah Hill Harmo...


In [132]:
final_df = pd.concat([old_df,df_21_20],ignore_index=True)

In [133]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb,Cast and crew
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...,NaN
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...,NaN
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...,NaN
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...,NaN
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...,NaN
...,...,...,...,...,...,...,...,...
6488,Kitao Sakurai,Eric André,NaN,Tiffany Haddish,Comedy,Bad Trip,NaN,Kitao Sakurai (director/screenplay); Dan Curry...
6489,Justin P. Lange,Guy Pearce,NaN,Stephen Lang,Horror,The Seventh Day,NaN,Justin P. Lange (director/screenplay); Guy Pea...
6490,Giorgio Serafini,William Shatner,NaN,Christopher Lloyd,Comedy Romance,Senior Moment,NaN,"Giorgio Serafini (director); Kurt Brungardt, C..."
6491,Mikey Alfred,Ryder McLaughlin,NaN,Miranda Cosgrove,Comedy Drama,North Hollywood,NaN,Mikey Alfred (director); Rusty Johnson (screen...


In [134]:
final_df.to_csv('main_data.csv',index=False)